In [1]:
import gzip
import torch
import json
import random

import numpy as np
import torch.nn.functional as F

from tqdm.notebook import tqdm
from random import randrange
from collections import defaultdict
from random import randrange, shuffle
from transformers import BertForNextSentencePrediction, AutoTokenizer

In [2]:
RANDOM_SEED = 42
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'

MAX_SEQUENCE_LENGTH = 256

MODEL_PATH = 'data/800k_100k_100k_best_model_state.pth'
BOOKS_PATH = 'data/top_200000_eng_books_filtered.json'
REVIEWS_PATH = 'data/goodreads_reviews_dedup.json.gz'

NUM_OF_USER_REVIEWS = 1000000
NUM_USERS_TO_EVALUATE = 100
K_MOST_SIMILAR_TO_CONSIDER = 10

In [3]:
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# User reviews data loading

In [4]:
def load_data(file_name, n):
    count = 0

    user_reviews = defaultdict(list)
    with gzip.open(file_name) as fin:
        for l in fin:
            review = json.loads(l)
            user_reviews[review['user_id']].append(review)
            count += 1
            
            # break if reaches the 100th line
            if count >= n:
                    break
    
    return user_reviews

In [5]:
reviews = load_data(REVIEWS_PATH, NUM_OF_USER_REVIEWS)
print(f'Number of unique users among the specified number of reviews: {len(reviews)}')

Number of unique users among the specified number of reviews: 20152


In [6]:
user_reviews = list(reviews.items())
user_reviews.sort(key=lambda pair: len(pair[1]), reverse=True)
user_reviews = user_reviews[:NUM_USERS_TO_EVALUATE]
for user_reviews_pair in user_reviews:
    shuffle(user_reviews_pair[1])

print(f'Maximum number of reviews for a single user among top {NUM_USERS_TO_EVALUATE} users with most ratings: {len(user_reviews[0][1])}')
print(f'Minimum number of reviews for a single user among top {NUM_USERS_TO_EVALUATE} users with most ratings: {len(user_reviews[NUM_USERS_TO_EVALUATE - 1][1])}')

Maximum number of reviews for a single user among top 100 users with most ratings: 5345
Minimum number of reviews for a single user among top 100 users with most ratings: 892


In [7]:
num_examples = 5
for idx in range(num_examples):
    print(f'{user_reviews[0][1][idx]}\n')

{'user_id': '8e7e5b546a63cb9add8431ee6914cf59', 'book_id': '22138435', 'review_id': '3f47f8fcd54034bbde53ec9d12d557c9', 'rating': 4, 'review_text': 'I liked it, the simple rhyme scheme, the graphic art style, the whole thing. \n Library copy', 'date_added': 'Thu Jan 22 05:57:11 -0800 2015', 'date_updated': 'Mon Feb 23 10:09:54 -0800 2015', 'read_at': 'Mon Feb 09 00:00:00 -0800 2015', 'started_at': 'Sun Feb 08 00:00:00 -0800 2015', 'n_votes': 0, 'n_comments': 0}

{'user_id': '8e7e5b546a63cb9add8431ee6914cf59', 'book_id': '7163889', 'review_id': '0940680fc4d04b912525833b4f27a93e', 'rating': 5, 'review_text': "Less than 24 hours after receiving our copies, the eldest and I have already finished it, and I've read the first 20 pages aloud to the KitKatPandaBatWolf. Oh, we love these books. And this morning I realized why. Unlike so many books for the new novel reader (those Daisy Meadows books, for example) there's a little bit of snark here. Not a lot. Just enough to keep the fantasy groun

# Books loading

In [8]:
with open(BOOKS_PATH, 'r') as books_file:
    books = json.load(books_file)

print(f'Loaded first {len(books)} books.')

Loaded first 200000 books.


In [9]:
filtered_user_reviews = []
for user_reviews_pair in user_reviews:
    filtered_reviews = [book_review for book_review in user_reviews_pair[1] if book_review['book_id'] in books]
    filtered_user_reviews.append((user_reviews_pair[0], filtered_reviews))
filtered_user_reviews.sort(key=lambda pair: len(pair[1]), reverse=True)

print(f'Maximum number of reviews for a single user among top {NUM_USERS_TO_EVALUATE} users with most ratings: {len(filtered_user_reviews[0][1])}')
print(f'Minimum number of reviews for a single user among top {NUM_USERS_TO_EVALUATE} users with most ratings: {len(filtered_user_reviews[NUM_USERS_TO_EVALUATE - 1][1])}')

Maximum number of reviews for a single user among top 100 users with most ratings: 816
Minimum number of reviews for a single user among top 100 users with most ratings: 11


In [10]:
num_examples = 5
for idx in range(num_examples):
    print(f'{filtered_user_reviews[0][1][idx]}\n')

{'user_id': '37b3e60b4e4152c580fd798d405150ff', 'book_id': '6839093', 'review_id': 'b66a3eefb08c74170fab1ef71c383615', 'rating': 3, 'review_text': "I'm really not thrilled with the book's central premise, but in the latter half of the volume, the story started building toward something more interesting that _might_ entice me to return.", 'date_added': 'Mon Jun 24 21:35:07 -0700 2013', 'date_updated': 'Mon Jun 24 21:35:36 -0700 2013', 'read_at': 'Wed Oct 19 00:00:00 -0700 2011', 'started_at': '', 'n_votes': 0, 'n_comments': 0}

{'user_id': '37b3e60b4e4152c580fd798d405150ff', 'book_id': '2108198', 'review_id': '6b6109fd53966b4e07b4bb4a0e20c46a', 'rating': 4, 'review_text': "A cute little book with nice Norse mythology. It's a little bit weak in the middle, but overall a good read.", 'date_added': 'Fri Jun 21 13:33:08 -0700 2013', 'date_updated': 'Fri Jun 21 13:33:34 -0700 2013', 'read_at': 'Tue Apr 06 00:00:00 -0700 2010', 'started_at': '', 'n_votes': 0, 'n_comments': 0}

{'user_id': '37

# Model and tokenizer loading

In [11]:
model = BertForNextSentencePrediction.from_pretrained(PRE_TRAINED_MODEL_NAME)
model.load_state_dict(torch.load(MODEL_PATH))
model = model.to(device)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Item based collaborative filtering evaluation

In [12]:
def calculate_books_similarity(book1_id, book2_id, books):
    book1, book2 = books[book1_id], books[book2_id]
    encoded_sequences = tokenizer(
        book1['description'],
        book2['description'],
        padding='longest',
        truncation='longest_first',
        return_tensors='pt',
        max_length=MAX_SEQUENCE_LENGTH
    )

    outputs = model(
        input_ids=encoded_sequences['input_ids'].to(device),
        attention_mask=encoded_sequences['attention_mask'].to(device),
        token_type_ids=encoded_sequences['token_type_ids'].to(device)
    )
    probs = F.softmax(outputs.logits, dim=1)

    # Class 0 notes if sent B follows sent A
    similarity_score = probs[0][0]
    return similarity_score.detach().item()

In [13]:
def evaluate_user(user_reviews_pair, books):
    user_id = user_reviews_pair[0]
    reviews = user_reviews_pair[1]

    user_ratings_torch = torch.tensor([review['rating'] for review in reviews], dtype=torch.float)
    baseline_rating = torch.mean(user_ratings_torch)

    # We want to estimate the rating of the first rated book
    # hence we get it and remove it from the rest of the reviews
    review_to_estimate = reviews[0]
    reviews = reviews[1:]
    similarities = []
    for review in tqdm(reviews):
        similarity = calculate_books_similarity(review_to_estimate['book_id'], review['book_id'], books)
        similarities.append((similarity, review['rating']))
    similarities.sort(key=lambda pair: pair[0], reverse=True)
    similarities = similarities[:K_MOST_SIMILAR_TO_CONSIDER]
    
    sims_torch = torch.tensor([sim for sim, _ in similarities], dtype=torch.float)
    ratings_torch = torch.tensor([real_rating for _, real_rating in similarities], dtype=torch.float)
    estimated_rating = torch.dot(sims_torch, ratings_torch) / torch.sum(sims_torch)

    print(f'Similarities and real ratings: {similarities}')
    print(f'Real rating: {review_to_estimate["rating"]}')
    print(f'Estimated rating: {estimated_rating}')
    print(f'Baseline rating(average for user): {baseline_rating}\n')

    return abs(review_to_estimate['rating'] - estimated_rating), abs(review_to_estimate['rating'] - baseline_rating)

In [14]:
estimated_rating_abs_diffs = []
baseline_rating_abs_diffs = []
for user_reviews_pair in tqdm(filtered_user_reviews):
    estimated_rating_abs_diff, baseline_rating_abs_diff = evaluate_user(user_reviews_pair, books)
    estimated_rating_abs_diffs.append(estimated_rating_abs_diff)
    baseline_rating_abs_diffs.append(baseline_rating_abs_diff)

mean_estimated_rating_abs_diff = torch.mean(torch.tensor(estimated_rating_abs_diffs))
print(f'Mean absolute rating - estimated rating difference: {mean_estimated_rating_abs_diff}')

baseline_estimated_rating_abs_diff = torch.mean(torch.tensor(baseline_rating_abs_diffs))
print(f'Mean absolute rating - baseline rating difference: {baseline_estimated_rating_abs_diff}')

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/815 [00:00<?, ?it/s]

Similarities and real ratings: [(0.9999223947525024, 4), (0.9999150037765503, 5), (0.9999055862426758, 5), (0.999904990196228, 4), (0.9999024868011475, 4), (0.9998922348022461, 4), (0.9998916387557983, 3), (0.999889612197876, 5), (0.9998887777328491, 4), (0.9998857975006104, 5)]
Real rating: 3
Estimated rating: 4.3000006675720215
Baseline rating(average for user): 3.878676414489746



  0%|          | 0/679 [00:00<?, ?it/s]

Similarities and real ratings: [(0.9999351501464844, 3), (0.9999350309371948, 4), (0.9999346733093262, 4), (0.9999301433563232, 5), (0.9999288320541382, 5), (0.9999279975891113, 4), (0.9999253749847412, 5), (0.9999208450317383, 2), (0.9999198913574219, 5), (0.9999198913574219, 5)]
Real rating: 5
Estimated rating: 4.1999993324279785
Baseline rating(average for user): 4.580882549285889



...

Similarities and real ratings: [(0.9999390840530396, 4), (0.9999344348907471, 3), (0.9804143905639648, 4), (0.002561979228630662, 5), (0.0011066041188314557, 4), (0.00038011546712368727, 3), (0.00020679301815107465, 4), (0.00015965610509738326, 4), (0.0001402519119437784, 3), (8.328935655299574e-05, 3)]
Real rating: 3
Estimated rating: 3.665661334991455
Baseline rating(average for user): 3.4473683834075928



  0%|          | 0/28 [00:00<?, ?it/s]

Similarities and real ratings: [(0.9999390840530396, 5), (0.9999319314956665, 5), (0.9996309280395508, 4), (0.9991301894187927, 3), (0.9967457056045532, 4), (0.9855087399482727, 3), (0.004883176181465387, 4), (0.0016299946000799537, 3), (0.0008543092990294099, 4), (0.0008046332513913512, 4)]
Real rating: 4
Estimated rating: 4.0022711753845215
Baseline rating(average for user): 3.689655065536499



  0%|          | 0/10 [00:00<?, ?it/s]

Similarities and real ratings: [(0.9998145699501038, 2), (0.9991549253463745, 3), (0.8661900758743286, 3), (0.00026199143030680716, 5), (0.00012720523227471858, 5), (6.72726528136991e-05, 4), (5.644989141728729e-05, 5), (5.2094546845182776e-05, 4), (5.134434104547836e-05, 1), (5.015620627091266e-05, 3)]
Real rating: 4
Estimated rating: 2.6514415740966797
Baseline rating(average for user): 3.545454502105713

Mean absolute rating - estimated rating difference: 0.8106637597084045
Mean absolute rating - baseline rating difference: 0.7889614105224609
